### This file runs the random trajectory analysis for plotting performance maps

In [1]:

import numpy as np
import pandas as pd
import lmfit

# run_constant is the simulator that runs the open loop experiment
from model_equations_and_simulators.run_constant import run_constant

# run_p_control is the simulator that runs the closed loop experiment with P control
from model_equations_and_simulators.run_p_control import run_p_control

# run_pi_control is the simulator that runs the closed loop experiment with PI control
from model_equations_and_simulators.run_pi_control import run_pi_control

# run_pi_control is the simulator that runs the closed loop experiment with PI control
from model_equations_and_simulators.run_pid_control import run_pid_control

# signal_analysis contains the functions to calculate the performance metrics
from signal_analysis import analyze_signal

In [2]:
# Load datasets for P control
data1 = pd.read_csv('experiment_data/P-FL_OD_run_data_042325.csv')
data2 = pd.read_csv('experiment_data/P-FL_OD_run_data_040625.csv')

# Extract relevant wells
P1_data1 = data1[['SP1_1', 'SP1_2', 'SP1_3', 'SP1_4', 'SP1_5', 'SP1_6']].to_numpy()
P2_data1 = data1[['SP2_1', 'SP2_2', 'SP2_3', 'SP2_4', 'SP2_5', 'SP2_6']].to_numpy()
P1_data2 = data2[['SP1_1', 'SP1_2', 'SP1_3']].to_numpy()
P2_data2 = data2[['SP2_1', 'SP2_2', 'SP2_3']].to_numpy()

green_data1 = data1[['G1']].to_numpy()
red_data1 = data1[['R1']].to_numpy()

green_data2 = data2[['G1', 'G2', 'G3']].to_numpy()
red_data2 = data2[['R1', 'R2', 'R3']].to_numpy()

# Time axis
interval = 10
min_len = min(P1_data1.shape[0], P1_data2.shape[0])
time_exp_P = np.arange(interval, (min_len + 1) * interval, interval)

# Truncate to matching length
green_data1 = green_data1[:min_len, :]
green_data2 = green_data2[:min_len, :]
red_data1 = red_data1[:min_len, :]
red_data2 = red_data2[:min_len, :]
P1_data1 = P1_data1[:min_len, :]
P1_data2 = P1_data2[:min_len, :]
P2_data1 = P2_data1[:min_len, :]
P2_data2 = P2_data2[:min_len, :]

# Combine datasets
green_combined = np.hstack([green_data1, green_data2])
red_combined = np.hstack([red_data1, red_data2])
P1_combined = np.hstack([P1_data1, P1_data2])
P2_combined = np.hstack([P2_data1, P2_data2])

# Compute mean profiles
green_mean_P = np.mean(green_combined, axis=1)
red_mean_P = np.mean(red_combined, axis=1)
P1_mean = np.mean(P1_combined, axis=1)
P2_mean = np.mean(P2_combined, axis=1)

# Use green max from experimental data
green_max_expt_P = np.max(green_mean_P)

# Setpoint values
st_pt_1 = 11500
st_pt_2 = 18500

# === Compute std for plotting shaded regions ===

# Standard deviations
P1_std = np.std(P1_combined, axis=1)
P2_std = np.std(P2_combined, axis=1)
green_std = np.std(green_combined, axis=1)
red_std = np.std(red_combined, axis=1)

# Normalize std
P1_std_norm = P1_std / green_max_expt_P
P2_std_norm = P2_std / green_max_expt_P
green_std_norm = green_std / green_max_expt_P
red_std_norm = red_std / green_max_expt_P

# Also normalize mean values for plotting
P1_mean_norm = P1_mean / green_max_expt_P
P2_mean_norm = P2_mean / green_max_expt_P
green_mean_norm = green_mean_P / green_max_expt_P
red_mean_norm = red_mean_P / green_max_expt_P


# Color dictionary
color_dict = {
    'set_point_1': '#6759d4',
    'set_point_2': '#ebab4b',
    'green': 'green',
    'red': 'red'
}

In [3]:
# Load dataset for PI control 

data1_PI = pd.read_csv('experiment_data/PI-FL_OD_run_data_042525.csv')
data2_PI = pd.read_csv('experiment_data/PI-FL_OD_run_data_041025.csv')

PI1_data1 = data1_PI[['SP1_1',  'SP1_2',  'SP1_3']].to_numpy()
PI2_data1 = data1_PI[['SP2_1',  'SP2_2',  'SP2_3']].to_numpy()

PI1_data2 = data2_PI[['SP1_1',  'SP1_2',  'SP1_3']].to_numpy()
PI2_data2 = data2_PI[['SP2_1',  'SP2_2',  'SP2_3']].to_numpy()

# Extract Green/Red constants (with replicates for 040825)
red_data1_PI = data1_PI[['R1',  'R2',  'R3']].to_numpy()
green_data1_PI = data1_PI[['G1',  'G2', 'G3']].to_numpy()

# For 0401025, Green and Red have only 1 well each
green_data2_PI = data2_PI[['G1']].to_numpy()
red_data2_PI = data2_PI[['R1']].to_numpy()

# Truncate all data to shared minimum length
min_len_PI = min(PI1_data1.shape[0], PI1_data2.shape[0])
time_exp_PI = np.arange(interval, (min_len_PI + 1) * interval, interval)
P1_data1_PI = PI1_data1[:min_len_PI]; P2_data1_PI = PI2_data1[:min_len_PI]
P1_data2_PI = PI1_data2[:min_len_PI]; P2_data2_PI = PI2_data2[:min_len_PI]
green_data1_PI = green_data1_PI[:min_len_PI]; green_data2_PI = green_data2_PI[:min_len_PI]
red_data1_PI = red_data1_PI[:min_len_PI]; red_data2_PI = red_data2_PI[:min_len_PI]

# Merge all experimental replicates
P1_combined_PI = np.hstack((P1_data1_PI, P1_data2_PI))
P2_combined_PI = np.hstack((P2_data1_PI, P2_data2_PI))
green_combined_PI = np.hstack((green_data1_PI, green_data2_PI))
red_combined_PI = np.hstack((red_data1_PI, red_data2_PI))

# Compute means and standard deviations
P1_mean_PI = np.mean(P1_combined_PI, axis=1)
P1_std_PI = np.std(P1_combined_PI, axis=1)
P2_mean_PI = np.mean(P2_combined_PI, axis=1)
P2_std_PI = np.std(P2_combined_PI, axis=1)
green_mean_PI = np.mean(green_combined_PI, axis=1)
green_std_PI = np.std(green_combined_PI, axis=1)
red_mean_PI = np.mean(red_combined_PI, axis=1)
red_std_PI = np.std(red_combined_PI, axis=1)

# Use green max from experimental data
green_max_expt_PI = np.max(green_mean_PI)

# Normalize std
P1_std_norm_PI = P1_std_PI / green_max_expt_P
P2_std_norm_PI = P2_std_PI / green_max_expt_P
green_std_norm_PI = green_std_PI / green_max_expt_P
red_std_norm_PI = red_std_PI / green_max_expt_P

# Also normalize mean values for plotting
P1_mean_norm_PI = P1_mean_PI / green_max_expt_P
P2_mean_norm_PI = P2_mean_PI / green_max_expt_P
green_mean_norm_PI = green_mean_PI / green_max_expt_P
red_mean_norm_PI = red_mean_PI / green_max_expt_P

In [4]:
# Load dataset for PID control 

data1_PID = pd.read_csv('experiment_data/PID-FL_OD_run_data_050725.csv')
data2_PID = pd.read_csv('experiment_data/PID-FL_OD_run_data_050925.csv')

PID1_data1 = data1_PID[['SP1_1',  'SP1_2',  'SP1_3']].to_numpy()
PID2_data1 = data1_PID[['SP2_1',  'SP2_2',  'SP2_3']].to_numpy()

PID1_data2 = data2_PID[['SP1_1',  'SP1_2',  'SP1_3']].to_numpy()
PID2_data2 = data2_PID[['SP2_1',  'SP2_2',  'SP2_3']].to_numpy()

red_data1_PID = data1_PID[['R1',  'R2',  'R3']].to_numpy()
green_data1_PID = data1_PID[['G1',  'G2', 'G3']].to_numpy()

red_data2_PID = data2_PID[['R1',  'R2',  'R3']].to_numpy()
green_data2_PID = data2_PID[['G1',  'G2', 'G3']].to_numpy()

# Truncate all data to shared minimum length
min_len_PID = min(PID1_data1.shape[0], PID1_data2.shape[0])
time_exp_PID = np.arange(interval, (min_len_PID + 1) * interval, interval)
P1_data1_PID = PID1_data1[:min_len_PID]; P2_data1_PID = PID2_data1[:min_len_PID]
P1_data2_PID = PID1_data2[:min_len_PID]; P2_data2_PID = PID2_data2[:min_len_PID]
green_data1_PID = green_data1_PID[:min_len_PID]; green_data2_PID = green_data2_PID[:min_len_PID]
red_data1_PID = red_data1_PID[:min_len_PID]; red_data2_PID = red_data2_PID[:min_len_PID]

# Merge all experimental replicates
P1_combined_PID = np.hstack((P1_data1_PID, P1_data2_PID))
P2_combined_PID = np.hstack((P2_data1_PID, P2_data2_PID))
green_combined_PID = np.hstack((green_data1_PID, green_data2_PID))
red_combined_PID = np.hstack((red_data1_PID, red_data2_PID))

# Compute means and standard deviations
P1_mean_PID = np.mean(P1_combined_PID, axis=1)
P1_std_PID = np.std(P1_combined_PID, axis=1)
P2_mean_PID = np.mean(P2_combined_PID, axis=1)
P2_std_PID = np.std(P2_combined_PID, axis=1)
green_mean_PID = np.mean(green_combined_PID, axis=1)
green_std_PID = np.std(green_combined_PID, axis=1)
red_mean_PID = np.mean(red_combined_PID, axis=1)
red_std_PID = np.std(red_combined_PID, axis=1)

# Use green max from experimental data
green_max_expt_PID = np.max(green_mean_PID)

# Normalize std
P1_std_norm_PID = P1_std_PID / green_max_expt_P
P2_std_norm_PID = P2_std_PID / green_max_expt_P
green_std_norm_PID = green_std_PID / green_max_expt_P
red_std_norm_PID = red_std_PID / green_max_expt_P

# Also normalize mean values for plotting
P1_mean_norm_PID = P1_mean_PID / green_max_expt_P
P2_mean_norm_PID = P2_mean_PID / green_max_expt_P
green_mean_norm_PID = green_mean_PID / green_max_expt_P
red_mean_norm_PID = red_mean_PID / green_max_expt_P

In [5]:
# Combine all the constant green/red data
min_len = 102

# Combine based on  all three data sets
green_combined = np.hstack([green_data1[:min_len], green_data2[:min_len], green_data1_PI[:min_len], green_data2_PI[:min_len], green_data1_PID[:min_len], green_data2_PID[:min_len]])
red_combined = np.hstack([red_data1[:min_len], red_data2[:min_len], red_data1_PI[:min_len], red_data2_PI[:min_len], red_data1_PID[:min_len], red_data2_PID[:min_len]])

green_mean_combined = np.mean(green_combined, axis=1)
green_std_combined = np.std(green_combined, axis=1)
green_max_combined = np.max(green_mean_combined)

red_mean_combined = np.mean(red_combined, axis=1)
red_std_combined = np.std(red_combined, axis=1)
red_max_combined = np.max(red_mean_combined)

In [6]:
def simulate_pid_trajectories(params, x0, st_pt_1, st_pt_2, green_max_expt_P, t_final = 960):
    # Determine green_max_model for normalization
    _, _, sol_green = run_constant(t_final, x0, params, constant_input='green')
    green_max_model = np.max(sol_green[:, 8])

    scaling_factor = (green_max_expt_P / green_max_model) * (1/60) 

    # Convert setpoints to model scale
    st_pt_1_model = st_pt_1 * (green_max_model / green_max_expt_P)
    st_pt_2_model = st_pt_2 * (green_max_model / green_max_expt_P)

    # Gains
    p_gain = 0.064 * scaling_factor
    pi_gain = 0.013 * scaling_factor
    pid_gain = 0.032 * scaling_factor
    tau_I = 1500
    tau_D = 120

    # Simulations for each controller
    t_p1_P, pctrl_1_P, _ = run_p_control(t_final, st_pt_1_model, p_gain, x0, params)
    t_p2_P, pctrl_2_P, _ = run_p_control(t_final, st_pt_2_model, p_gain, x0, params)

    t_p1_PI, pctrl_1_PI, _, _ = run_pi_control(t_final, st_pt_1_model, pi_gain, tau_I, x0, params)
    t_p2_PI, pctrl_2_PI, _, _ = run_pi_control(t_final, st_pt_2_model, pi_gain, tau_I, x0, params)

    t_p1_PID, pctrl_1_PID, _, _ = run_pid_control(t_final, st_pt_1_model, pid_gain, tau_I, tau_D, x0, params)
    t_p2_PID, pctrl_2_PID, _, _ = run_pid_control(t_final, st_pt_2_model, pid_gain, tau_I, tau_D, x0, params)

    # Normalize
    normalize = lambda x: np.array(x) / green_max_model
    data = {
        'P': {'SP1': (t_p1_P, normalize(pctrl_1_P)), 'SP2': (t_p2_P, normalize(pctrl_2_P))},
        'PI': {'SP1': (t_p1_PI, normalize(pctrl_1_PI)), 'SP2': (t_p2_PI, normalize(pctrl_2_PI))},
        'PID': {'SP1': (t_p1_PID, normalize(pctrl_1_PID)), 'SP2': (t_p2_PID, normalize(pctrl_2_PID))}
    }
    return data


In [7]:
def generate_random_parameters(params, n_samples = 50, sampling_fraction = 0.15):
    """
    Generate random parameter sets within ±sampling_fraction bounds.
    
    Returns:
    - A list of lmfit.Parameter objects (clones with randomized values).
    """
    np.random.seed(11) # fixing for uniform sampling and replicability
    random_sets = []
    for _ in range(n_samples):
        p_copy = params.copy()
        for name, param in params.items():
            if not param.vary:
                continue
            nominal = param.value
            delta = nominal * sampling_fraction
            new_value = np.random.uniform(nominal - delta, nominal + delta)
            p_copy[name].value = new_value
        random_sets.append(p_copy)
    return random_sets

In [8]:
# define model parameters 


p = pd.read_csv('parameters/TCS_model_param_file.csv').to_numpy()
p = p[:,2]

params = lmfit.Parameters()

params.add(name = 'k_green', value = p[0], min = 1e-4, max = 1e4, vary = 1)
params.add(name = 'k_red', value = p[1], min = 1e0, max = 1e4, vary = 1)
params.add(name = 'b_green', value = p[2], min = 1e-4, max = 1e4, vary = 1)
params.add(name = 'b_red', value = p[3], min = 1e0, max = 1e4, vary = 1)

params.add(name = 'k_sp_b', value = p[4], min = 1e-4, max = 1e4, vary = 1)
params.add(name = 'k_sp_u', value = p[5], min = 1e-4, max = 1e4, vary = 1)

params.add(name = 'k_rp_b', value = p[6], min = 1e-4, max = 1e4, vary = 1)
params.add(name = 'k_rp_u', value = p[7], min = 1e-4, max = 1e4, vary = 1)

params.add(name = 'beta', value = p[8], min = 1e-1, max = 500, vary = 1)
params.add(name = 'l0', value = p[9], min = 0, max = 0.5, vary = 1)
params.add(name = 'Kc', value = p[10], min = 1e-4, max = 1e4, vary = 1)
params.add(name = 'd_m', value = p[11], min = 0.05, max = 0.35, vary = 1)
params.add(name = 'k_tl', value = p[12], min = 0.05, max = 10, vary = 1)
params.add(name = 'k_tli_b', value = p[13], min = 1e-4, max = 1e4, vary = 1)
params.add(name = 'k_tli_u', value = p[14], min = 1e-4, max = 1e4, vary = 1)
params.add(name = 'd_p', value = p[15], min = 1e-6, max = 1e-1, vary = 1)
params.add(name = 'k_fold', value = p[16], min = 0.05, max = 0.3, vary = 1)
params.add(name = 'b_fold', value = p[17], min = 0.1, max = 2, vary = 1)
params.add(name = 'n_gamma', value = p[18], min = 0.05, max = 0.9, vary = 1)
params.add(name = 'R_max', value = p[19], min = 1e0, max = 1e4, vary = 1)

params.add(name = 'S_0', value = p[20], min = 1e-4, max = 1e4, vary = 1)
params.add(name = 'R_0', value = p[21], min = 1e-4, max = 1e4, vary = 1)
params.add(name = 'Sp_0', value = p[22], min = 1e-4, max = 1e4, vary = 1)
params.add(name = 'Rp_0', value = p[23], min = 1e-4, max = 1e4, vary = 1)
params.add(name = 'mRNA_0', value = p[24], min = 1e-4, max = 1e4, vary = 1)
params.add(name = 'P_0', value = p[25], min = 1e-4, max = 1e4, vary = 1)
params.add(name = 'Pm_0', value = p[26], min = 1e-4, max = 1e4, vary = 1)


params.add(name = 'k_gr', value = p[27], vary = 0)
params.add(name = 'C_max', value = p[28], vary = 0)
params.add(name = 'C_0', value = p[29], vary = 0)

params.add(name = 'n_tcs', value = p[30], min = 0.1, max = 5, vary = 1)



# define initial conditions and solve the ODEs

x0 = np.zeros(11)

x0[0] = p[20] # S
x0[1] = p[22]# Sp
x0[2] = p[21] # R
x0[3] = p[23] # Rp
x0[4] = 0 # Ac
x0[5] = p[24] # mRNA
x0[6] = 0 # Ctic
x0[7] = p[25] # Unfolded Protein 
x0[8] = p[26] # Folded Protein 
x0[9] = p[29] # Initial Cell count
x0[10] = 0 # Initial intergal error

# Total time of simulation
t_final = 960

In [9]:
# ----------------------------
# Monte Carlo sampling + metrics
# ----------------------------
n_samples = 50
interval_percentage = 0.15
tolerance_percentage = 0.1

random_param_sets = generate_random_parameters(params, n_samples=n_samples, sampling_fraction=interval_percentage)

# For normalization in metrics
green_mean_norm = green_mean_combined / green_max_combined
red_mean_norm = red_mean_combined / green_max_combined
max_dyn_range = float(np.max(green_mean_norm - red_mean_norm))
tol_band = tolerance_percentage * max_dyn_range

all_metrics_rows = []
for i, param_set in enumerate(random_param_sets):
    print(f"Simulating for perturbation {i+1}/{n_samples}...")
    sim_data = simulate_pid_trajectories(param_set, x0, st_pt_1, st_pt_2, green_max_expt_P)

    for sp_label, target in [('SP1', st_pt_1), ('SP2', st_pt_2)]:
        st_norm = target / green_max_expt_P
        for ctrl in ['P', 'PI', 'PID']:
            t, s = sim_data[ctrl][sp_label]
            rise_time, settling_time, offset_error = analyze_signal(
                s, t, st_norm,
                tolerance_band=tol_band,
                max_dynamic_range=max_dyn_range,
                settle_time_cap=1000
            )
            all_metrics_rows.append({
                'Settling Time': settling_time,
                'Offset Error (%)': offset_error,
                'Control': ctrl,
                'Perturbation': i,
                'Set Point': sp_label
            })

all_metrics = pd.DataFrame(all_metrics_rows)

# ----------------------------
# Save for plotting
# ----------------------------
out_csv = 'metrics_data/all_metrics_random_sampling.csv'
all_metrics.to_csv(out_csv, index=False)
print(f"Saved metrics to {out_csv}")

Simulating for perturbation 1/50...
Simulating for perturbation 2/50...
Simulating for perturbation 3/50...
Simulating for perturbation 4/50...
Simulating for perturbation 5/50...
Simulating for perturbation 6/50...
Simulating for perturbation 7/50...
Simulating for perturbation 8/50...
Simulating for perturbation 9/50...
Simulating for perturbation 10/50...
Simulating for perturbation 11/50...
Simulating for perturbation 12/50...
Simulating for perturbation 13/50...
Simulating for perturbation 14/50...
Simulating for perturbation 15/50...
Simulating for perturbation 16/50...
Simulating for perturbation 17/50...
Simulating for perturbation 18/50...
Simulating for perturbation 19/50...
Simulating for perturbation 20/50...
Simulating for perturbation 21/50...
Simulating for perturbation 22/50...
Simulating for perturbation 23/50...
Simulating for perturbation 24/50...
Simulating for perturbation 25/50...
Simulating for perturbation 26/50...
Simulating for perturbation 27/50...
Simulating